# Kerchunk

This is a companion notebook to [Kerchunk and Pangeo-Forge](notebooks/kerchunk-pangeo-forge.ipynb) to demonstrate how to recreate the processing pipeline using `Kerchunk` without `Pangeo-Forge`. Details on the dataset and processing steps are outlined in more detail in the previous notebook.

More detailed guides to `Kerchunk` can be found in the [Docs](https://fsspec.github.io/kerchunk/) and the [Project Pythia Cookbook](https://projectpythia.org/kerchunk-cookbook/README.html)

In [1]:
import dask
import fsspec
import glob
import logging
import ujson
import warnings
import xarray as xr

from tempfile import TemporaryDirectory
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

## Exaimine a Single File

In [2]:
import xarray as xr

ds = xr.open_dataset(
    "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_2021.nc"
)

In [3]:
ds

<xarray.Dataset>
Dimensions:          (day: 365, lat: 585, crs: 1, lon: 1386)
Coordinates:
  * day              (day) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-12-31
  * lat              (lat) float64 49.4 49.36 49.32 49.28 ... 25.15 25.11 25.07
  * crs              (crs) float32 3.0
  * lon              (lon) float64 -124.8 -124.7 -124.7 ... -67.14 -67.1 -67.06
Data variables:
    burning_index_g  (day, lat, lon) float32 ...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       30 March 2023
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

### Startup a Dask Client for Parallel Processing

In [4]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59850,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:59871,Total threads: 1
Dashboard: http://127.0.0.1:59876/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:59853,


### Create List of URLs

In [28]:
start_year = 1979
end_year = 1981  # Data is available until 2022. This example only processes 2 years

years = list(range(start_year, end_year + 1))

url_list = [
    f"http://www.northwestknowledge.net/metdata/data/bi_{time}.nc" for time in years
]

### Create a Temp Directory to Write

In [29]:
import os
from tempfile import TemporaryDirectory

td = TemporaryDirectory()
target_root = td.name
target_root = "esip"
store_name = "kerchunk"
target_store = os.path.join(target_root, store_name)

### Use `SingleHdf5ToZarr` to Create Kerchunk References


In [30]:
def generate_json_reference(fil, output_dir: str):
    h5chunks = SingleHdf5ToZarr(fil, inline_threshold=300)
    fname = fil.split("/")[-1].strip(".nc")
    outf = f"{output_dir}/{fname}.json"
    with open(outf, "wb") as f:
        f.write(ujson.dumps(h5chunks.translate()).encode())
    return outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil, target_store) for fil in url_list]

In [31]:

%%time 
# Start parallel processing

warnings.filterwarnings("ignore")
dask.compute(tasks)

(['esip/kerchunk/bi_1979.json', 'esip/kerchunk/bi_1980.json'],)

### Combine Kerchunk Reference Files into Combined Reference

In [36]:
%%time

# Create a list of reference json files
output_files = glob.glob(f"{target_store}/*.json")

# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(
    output_files,
    concat_dims=["day"],
    identical_dims=["lat", "lon", "crs"],
)

multi_kerchunk = mzz.translate()

# Write in-memory Kerchunk combined reference to json
output_fname = "Combined_Reference.json"
with open(f"{output_fname}", "wb") as f:
    f.write(ujson.dumps(multi_kerchunk).encode())

CPU times: user 590 ms, sys: 103 ms, total: 693 ms
Wall time: 2.59 s


## Load Kerchunk Dataset

In [34]:
fs = fsspec.filesystem(
    "reference",
    fo="Combined_Reference.json",
    remote_protocol="http",
    remote_options={"anon": True},
    skip_instance_cache=True,
)
ds = xr.open_dataset(fs.get_mapper(""), engine="zarr")

In [35]:
ds

<xarray.Dataset>
Dimensions:          (day: 16071, lat: 585, lon: 1386, crs: 1)
Coordinates:
  * crs              (crs) uint16 3
  * day              (day) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
  * lat              (lat) float64 0.0 2.16e-314 0.0 ... 4.832e+276 4.832e+276
  * lon              (lon) float64 0.0 2.16e-314 0.0 ... 1.622e+213 1.522e+171
Data variables:
    burning_index_g  (day, lat, lon) float32 ...
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...